In [16]:
import pandas as pd
from datetime import datetime
from datetime import timedelta                                   

In [17]:
NGR=pd.read_csv('playerstats.csv')

In [18]:
# Deletando a última linha do DataFrame
NGR = NGR.drop(NGR.index[-1])

In [19]:
NGR["First deposit"] =pd.to_datetime(NGR["First deposit"], errors = 'coerce')

In [20]:
NGR=NGR.loc[(NGR["Advertiser"]!="crmbonus") & (NGR["Advertiser"]!="pine10")]

In [21]:
NGR["Advertiser"].unique()

array(['ally0034', 'ally0004', 'ally0011', 'ally0032', 'ally0005',
       'ally0002', 'rubju300', 'ally0031', 'jarbasbezerra05', 'ally0036',
       'rspn1', 'jafe1000', 'expx8030', 'ally0037', 'ally0012',
       'ally0023', 'ally0040', 'franber', 'thago', 'ally0003', 'jompa',
       'ally0022', 'rspn5', 'rspn10', 'jeh50', 'Gn8383', 'ally0001',
       'rspn7', 'ally0018', 'ally0029', 'ally0015', 'rspn2', 'ally0007',
       'sostenes1', 'ally0009', 'biar16', 'sixpoint02', 'asem01',
       'pafeit', 'adriano08'], dtype=object)

In [22]:
NGR = NGR.loc[(NGR["First deposit"].dt.year == 2024)]

In [23]:
NGR["Calculated_NGR"] = NGR['Casino Real Bets'] - NGR['Casino Real Wins'] -NGR['Redeemed bonus initial amounts'] - NGR['Redeemed bonus pending winnings']

In [24]:
NGR['First deposit date abreviated'] =NGR["First deposit"].dt.strftime('%m/%Y')

In [25]:
NGR.columns.values

array(['Username', 'Country', 'Advertiser', 'Signed up', 'First deposit',
       'Last deposit date', 'Wants mail', 'Deposits', 'Deposit count',
       'Average deposit', 'Approved Withdrawals',
       'Approved Withdrawal count', 'Casino NGR', 'Casino bonuses',
       'Casino bonus count', 'Redeemed bonus initial amounts',
       'Redeemed bonus pending winnings', 'Poker bonus reverses',
       'Casino progressive share', 'Casino progressive wins',
       'Casino Bets', 'Casino Wins', 'Casino Real Bets',
       'Casino Real Wins', 'Casino game count', 'Casino bonuses removed',
       'Frozen', 'Sportsbook wins', 'Sportsbook bets',
       'Sportsbook gross income', 'Calculated_NGR',
       'First deposit date abreviated'], dtype=object)

In [26]:
output = NGR.groupby('First deposit date abreviated')[["Calculated_NGR","Deposits","Deposit count"]].sum().reset_index()

In [27]:
output

,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,3037.47,3831.00,23
1,02/2024,6070.38,7118.04,119
2,03/2024,181.07,8291.60,108
3,04/2024,2281.49,6237.93,138
4,05/2024,10104.74,24661.18,366
5,06/2024,7416.16,38035.26,586
6,07/2024,26508.03,42084.63,598
7,08/2024,7015.77,75130.18,1095
8,09/2024,27549.07,85387.31,1168
9,10/2024,25245.57,123623.31,2071


In [28]:
output.loc[len(output)] = ["Total",output["Calculated_NGR"].sum(),output["Deposits"].sum(),output["Deposit count"].sum()]
output


,First deposit date abreviated,Calculated_NGR,Deposits,Deposit count
0,01/2024,3037.47,3831.00,23
1,02/2024,6070.38,7118.04,119
2,03/2024,181.07,8291.60,108
3,04/2024,2281.49,6237.93,138
4,05/2024,10104.74,24661.18,366
5,06/2024,7416.16,38035.26,586
6,07/2024,26508.03,42084.63,598
7,08/2024,7015.77,75130.18,1095
8,09/2024,27549.07,85387.31,1168
9,10/2024,25245.57,123623.31,2071


In [29]:
import requests
# Conversão para DataFrame
df = pd.DataFrame(output)

# Renomeando colunas
df.rename(columns={
    "First deposit date abreviated": "Mês",
    "Calculated_NGR": "NGR",
    "Deposits": "Depósitos"
}, inplace=True)

# Fazendo a requisição para a API de câmbio (exemplo com Exchange Rates API)
api_key = "YOUR_API_KEY"  # Substitua pela sua chave de API
url = f"https://api.exchangerate-api.com/v4/latest/BRL"
response = requests.get(url)
data = response.json()

# Obter a taxa de conversão de BRL para USD
exchange_rate = data["rates"]["USD"]

# Aplicando a conversão e formatando
df["NGR_USD"] = df["NGR"] * exchange_rate
df["Depósitos_USD"] = df["Depósitos"] * exchange_rate

# Formatando os valores em R$ e em US$
df["NGR"] = df["NGR"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos"] = df["Depósitos"].apply(lambda x: f'R${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["NGR_USD"] = df["NGR_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))
df["Depósitos_USD"] = df["Depósitos_USD"].apply(lambda x: f'${x:,.2f}'.replace(",", "X").replace(".", ",").replace("X", "."))

df['Data'] = "15/11/2024"
df['Ordem'] = range(1, len(df) + 1)


df

,Mês,NGR,Depósitos,Deposit count,NGR_USD,Depósitos_USD,Data,Ordem
0,01/2024,"R$3.037,47","R$3.831,00",23,"$525,48","$662,76",15/11/2024,1
1,02/2024,"R$6.070,38","R$7.118,04",119,"$1.050,18","$1.231,42",15/11/2024,2
2,03/2024,"R$181,07","R$8.291,60",108,"$31,33","$1.434,45",15/11/2024,3
3,04/2024,"R$2.281,49","R$6.237,93",138,"$394,70","$1.079,16",15/11/2024,4
4,05/2024,"R$10.104,74","R$24.661,18",366,"$1.748,12","$4.266,38",15/11/2024,5
5,06/2024,"R$7.416,16","R$38.035,26",586,"$1.283,00","$6.580,10",15/11/2024,6
6,07/2024,"R$26.508,03","R$42.084,63",598,"$4.585,89","$7.280,64",15/11/2024,7
7,08/2024,"R$7.015,77","R$75.130,18",1095,"$1.213,73","$12.997,52",15/11/2024,8
8,09/2024,"R$27.549,07","R$85.387,31",1168,"$4.765,99","$14.772,00",15/11/2024,9
9,10/2024,"R$25.245,57","R$123.623,31",2071,"$4.367,48","$21.386,83",15/11/2024,10


In [30]:
import uuid

# Carregar o arquivo CSV existente
df_existing = pd.read_excel("daily_playerstats.xlsx")

# Filtrar as colunas necessárias (incluindo 'ID')
df_existing = df_existing[["Mês", "NGR", "Depósitos", "Deposit count", "NGR_USD", "Depósitos_USD", "Data", "ID", "Ordem"]]

# Gerar um ID único para toda a nova tabela 'final_result'
unique_id = str(uuid.uuid4())

# Adicionar a mesma ID para todas as linhas da nova tabela
df['ID'] = unique_id

# Concatenar o DataFrame novo (final_result) ao existente
df_combined = pd.concat([df_existing, df], ignore_index=True)

# Remover duplicatas com base na coluna "Username" (ou outra coluna de sua escolha)
#df_combined = df_combined.drop_duplicates(subset=["Ordem"], keep="last")

# Salvar o DataFrame combinado e sem duplicatas no arquivo CSV
df_combined.to_excel("daily_playerstats.xlsx",index=False)